<a href="https://colab.research.google.com/github/caiovidaln/ri-lab-4/blob/master/ri_lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import nltk
import time
import heapq
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

_stopwords = set(stopwords.words('portuguese'))

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Carregando Coleção

In [46]:
data = pd.read_csv("./results.csv")
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


# Questão 1

In [47]:

tokenData = pd.DataFrame(data['text'], columns=['text'])
tokenData['tokenized_text'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)
tokenData.head()

,text,tokenized_text
0,A juíza federal Ivani Silva da Luz de Brasíli...,"[a, juíza, federal, ivani, silva, da, luz, de,..."
1,Menos de 24 horas depois de a juíza federal Iv...,"[menos, de, 24, horas, depois, de, a, juíza, f..."
2,Quando determinou que de 31 de março 1964 u...,"[quando, determinou, que, de, 31, de, março, 1..."
3,Quando era pequeno Luisa e Gabriel se preo...,"[quando, era, pequeno, luisa, e, gabriel, se, ..."
4,Desde a manhã da última segunda-feira e ...,"[desde, a, manhã, da, última, segunda-feira, e..."


In [0]:
def is_tokenized_word(word):

  return len(word) > 1 and any(c.isalnum() for c in word)

In [0]:
def build_index():
  index = {}
  num_document = 0

  for text in tokenData.tokenized_text:
    num_document += 1
    words_set = set(text)
    for word in words_set:
      if is_tokenized_word(word) and word.lower() not in _stopwords:
        if word not in index.keys():
          index[word] = []
        index[word].append((num_document, text.count(word)))

  return index

In [50]:
indexed_data = pd.DataFrame()
index = build_index()
indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents (With Count)'] = [docs for docs in index.values()]
indexed_data.to_csv('indexed_data.csv', encoding='utf-8', index=False)
indexed_data.head(10)

,Word,Documents (With Count)
0,jornalista,"[(1, 1), (4, 2), (6, 3), (7, 1), (9, 1), (11, ..."
1,golpistas,"[(1, 2)]"
2,1964.,"[(1, 1), (25, 2), (208, 1), (218, 1)]"
3,ruptura,"[(1, 1), (9, 1), (25, 1), (65, 1), (83, 1), (1..."
4,uso,"[(1, 1), (2, 1), (11, 1), (12, 2), (18, 1), (1..."
5,sentença,"[(1, 3), (103, 1), (106, 1), (113, 1), (169, 1..."
6,preferem,"[(1, 1), (63, 1), (65, 1), (125, 1), (172, 1),..."
7,desta,"[(1, 1), (2, 1), (20, 2), (23, 4), (26, 1), (2..."
8,movimento,"[(1, 1), (6, 1), (10, 1), (11, 9), (18, 1), (2..."
9,repetir,"[(1, 1), (76, 1), (83, 1), (151, 1)]"


# Questão 2

In [0]:
def document_at_time(query, index, k):
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_pos in range(1, len(data) + 1):
        sd = 0
        for document_count in document_count_list:
            for i in document_count:
                if (i[0] == document_pos):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, document_pos))
    
    return heapq.nlargest(k, r)
  
def term_at_time(query, index, k):
    a = {}
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_count in document_count_list:
        for post in document_count:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))

    return heapq.nlargest(k, r)

In [52]:
queries = ["justiça", "bolsa", "mercado", "avião", "país"]
k = 10
results_doc = []
results_term = []

for query in queries:
  score_doc = document_at_time(query, index, k)
  results_doc.append(score_doc)
  score_term = term_at_time(query, index, k)
  results_term.append(score_term)

queries_df = pd.DataFrame()
queries_df['Consulta'] = queries
queries_df['Documento por vez'] = results_doc
queries_df['Termo por vez'] = results_term
queries_df['Mesmo resultado'] = queries_df['Documento por vez'] == queries_df['Documento por vez']
queries_df

,Consulta,Documento por vez,Termo por vez,Mesmo resultado
0,justiça,"[(7, 66), (6, 162), (4, 166), (4, 29), (3, 217...","[(7, 66), (6, 162), (4, 166), (4, 29), (3, 217...",True
1,bolsa,"[(9, 203), (3, 215), (3, 142), (3, 138), (2, 2...","[(9, 203), (3, 215), (3, 142), (3, 138), (2, 2...",True
2,mercado,"[(8, 125), (8, 108), (3, 127), (3, 22), (2, 18...","[(8, 125), (8, 108), (3, 127), (3, 22), (2, 18...",True
3,avião,"[(7, 143), (3, 224), (2, 89), (1, 247), (1, 24...","[(7, 143), (3, 224), (2, 89), (1, 247), (1, 24...",True
4,país,"[(18, 151), (11, 173), (11, 166), (8, 208), (6...","[(18, 151), (11, 173), (11, 166), (8, 208), (6...",True


In [53]:
import time
        
def document_at_time_1000_times_per_query():
  for query in queries:
    for _ in range(10000):
      document_at_time(query, index, k)

def term_at_time_1000_times_per_query():
  for query in queries:
    for _ in range(1000):
      term_at_time(query, index, k)
      
qty_queries_execution = 1000 * 5

document_start = time.time()
document_at_time_1000_times_per_query()
document_time_elapsed = time.time() - document_start
document_average_time = document_time_elapsed / qty_queries_execution
print("Tempo médio para execução de Documento Por Vez: ", document_average_time)

term_start = time.time()
term_at_time_1000_times_per_query()
term_time_elapsed = time.time() - term_start
term_average_time = term_time_elapsed / qty_queries_execution
print("Tempo médio para execução de Termo Por Vez: ", term_average_time)

Tempo médio para execução de Documento Por Vez:  0.008000185251235963
Tempo médio para execução de Termo Por Vez:  6.164608001708985e-05


O tempo médio para execução de "Termo Por Vez" foi consideravelmente menor que o de "Documento Por Vez", cerca de 400x mais rápido, como já era esperado.

# Questão 3

In [0]:
def conjuctive_query(query, indexes, k):
  query_indexes = []
  rank = []
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort()
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

In [55]:
conjuctives_queries = ["forças armadas", "prisão deputado", "donald trump", "companhias aéreas", "juros mercado"]

k = 10
conjuctives_results = []

for query in conjuctives_queries:
  conjuctive_score = conjuctive_query(query, index, k)
  conjuctives_results.append(conjuctive_score)

conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = conjuctives_queries
conj_queries_df['Results'] = conjuctives_results
conj_queries_df

,Query,Results
0,forças armadas,"[(15, 150), (9, 25), (8, 208), (7, 166), (6, 1..."
1,prisão deputado,"[(11, 204), (8, 202), (2, 225), (2, 207), (2, ..."
2,donald trump,"[(4, 211), (3, 145), (3, 105), (3, 89), (2, 24..."
3,companhias aéreas,"[(6, 143), (5, 134)]"
4,juros mercado,"[(4, 34), (2, 24)]"
